# Model Registration with VPC Config

In [1]:
import boto3
import sagemaker
from sagemaker.model import Model
from time import gmtime, strftime
session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket_name = 'sagemaker-mlops-demo-dev-2024'
model_package_group_name = 'mnist-classification-models'
sm_client = boto3.client('sagemaker')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:

#vpc configuration with multiple subnets
vpc_config = {
    'Subnets': [
        'subnet-08ab4f85521900ccb',
        'subnet-00ab8c58bb53cfbc9'
    ],
    'SecurityGroupIds': ['sg-061f2bba40ed5c109']
}

In [3]:
training_jobs = sm_client.list_training_jobs(
    StatusEquals='Completed',
    SortBy='CreationTime',
    SortOrder='Descending',
    MaxResults=1
)
latest_job_name = training_jobs['TrainingJobSummaries'][0]['TrainingJobName']
training_job = sm_client.describe_training_job(TrainingJobName=latest_job_name)
model_url = training_job['ModelArtifacts']['S3ModelArtifacts']
print(f"Latest training job found: {latest_job_name}")
print(f"Model location: {model_url}")

Latest training job found: tensorflow-training-2024-11-24-09-08-21-911
Model location: s3://sagemaker-mlops-demo-dev-2024/mnist/models/tensorflow-training-2024-11-24-09-08-21-911/output/model.tar.gz


In [4]:
#configure inference image
inference_image_uri = sagemaker.image_uris.retrieve(
    framework='tensorflow',
    region=session.boto_session.region_name,
    version='2.11.0',
    py_version='py39',
    instance_type='ml.m5.large',
    image_scope='inference'
)

In [5]:
#create and register model with vpc config
model = Model(
    image_uri=inference_image_uri,
    model_data=model_url,
    role=role,
    vpc_config=vpc_config,
    sagemaker_session=session,
    enable_network_isolation=True
)
model_package = model.register(
    model_package_group_name=model_package_group_name,
    approval_status='PendingManualApproval',
    content_types=["application/json"],
    response_types=["application/json"],
    inference_instances=['ml.m5.large'],
    transform_instances=['ml.m5.large'],
    model_metrics=None
)
print(f"\nModel registered with ARN: {model_package.model_package_arn}")
response = sm_client.describe_model_package(
    ModelPackageName=model_package.model_package_arn
)
print("\nModel registration status:")
print(f"Status: {response['ModelPackageStatus']}")
print(f"Approval Status: {response['ModelApprovalStatus']}")


Model registered with ARN: arn:aws:sagemaker:us-east-1:515966515324:model-package/mnist-classification-models/8

Model registration status:
Status: Completed
Approval Status: PendingManualApproval
